

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" /> 

# Laboratorio 5: Análisis Técnico y/o fundamental



### Equipo X

***Esteban Navarrete Garrido***


***José Alejandro Preciado Gandara***


***José Luis Suárez de Alba*** 


### Propósito: 

<div class=text-justify>  
Que puedas describir los efectos cuantitativos en el precio durante los comunicados de indicadores macroeconómicos, así como el detectar oportunidades de inversión a partir de los resultados obtenidos. Lo anterior, utilizando lo que se conoce como "Calendario Económico", y más en particular, aquel que incorpora información de las principales economías del mundo por lo que es ampliamiente utilizado en el mercado de Forex.
</div>

### Calendario económico:
&nbsp;
<div class=text-justify> 
**Retail salescontrol group:** Representa las ventas totales de la industria que se utilizan para preparar las estimaciones de los gastos del consumo personal para la mayoría de los productos
</div>

<div class=text-justify> 
**Retail sales ex Autos (MOM):** Representan los datos mensuales que muestran todos los productos vendidos por los minoristas en base a una muestra de tiendas minoristas de diferentes tipos y tamaños, excepto el sector automotriz.
</div>

### Importar librerias

In [19]:
import pandas as pd
import numpy as np
import datetime as datetime
import plotly.graph_objs as go
import chart_studio.plotly as py
from collections import Counter
from itertools import chain

ModuleNotFoundError: No module named 'chart_studio'

### Importar datos

In [24]:
precios = 'precios_historicos_eurusd.csv'
calendario = 'calendario_economico.csv'

In [25]:
precios_df = pd.read_csv(precios, header=0, sep=',', parse_dates = False, skip_blank_lines = True)
precios_df.head()

,timestamp,open,high,low,close
0,2009-01-06 05:00:00,1.35564,1.35565,1.35383,1.35439
1,2009-01-06 05:05:00,1.35443,1.35489,1.35424,1.35474
2,2009-01-06 05:10:00,1.35480,1.35552,1.35439,1.35539
3,2009-01-06 05:15:00,1.35545,1.35549,1.35101,1.35101
4,2009-01-06 05:20:00,1.35101,1.35130,1.35021,1.35086


In [26]:
ce_df = pd.read_csv(calendario, header=0,sep=',', parse_dates=False, skip_blank_lines=True)
ce_df.head()

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus
0,01/05/2009 09:30:00,EUR,Sentix Investor Confidence,European Monetary Union,1,-34.4,-42.3,NaN
1,01/05/2009 15:00:00,USD,Construction Spending (MoM),United States,1,-1.2,-0.4,-1.3
2,01/06/2009 08:55:00,EUR,Markit Services PMI,Germany,1,46.6,46.4,46.4
3,01/06/2009 09:00:00,EUR,Markit Services PMI,European Monetary Union,0,42.1,42.0,42.0
4,01/06/2009 10:00:00,EUR,Consumer Price Index (YoY),European Monetary Union,2,1.6,2.1,1.8


In [27]:
precios_df['timestamp'] = pd.to_datetime(precios_df['timestamp'])
precios_df['timestamp'] = precios_df['timestamp'].dt.tz_localize('UTC')
#precios_df.index = precios_df['timestamp'] ; precios_df = precios_df.drop('timestamp',axis=1)
precios_df.head()

,timestamp,open,high,low,close
0,2009-01-06 05:00:00+00:00,1.35564,1.35565,1.35383,1.35439
1,2009-01-06 05:05:00+00:00,1.35443,1.35489,1.35424,1.35474
2,2009-01-06 05:10:00+00:00,1.35480,1.35552,1.35439,1.35539
3,2009-01-06 05:15:00+00:00,1.35545,1.35549,1.35101,1.35101
4,2009-01-06 05:20:00+00:00,1.35101,1.35130,1.35021,1.35086


In [28]:
ce_df['timestamp'] = pd.to_datetime(ce_df['timestamp'])
ce_df['timestamp'] = ce_df['timestamp'].dt.tz_localize('UTC')
#ce_df.index = ce_df['timestamp'] ; ce_df = ce_df.drop('timestamp',axis=1)
ce_df.head()
ce_df.consensus.fillna(ce_df.previous, inplace= True)

In [29]:
Control = pd.DataFrame()
Autos   = pd.DataFrame()

for i in range(len(ce_df)):
    f = ce_df.index[i]
    if ce_df['Name'][i] == "Retail Sales Control Group":
        f=ce_df.index[i]
        Control[f] = ce_df.iloc[i]
    elif ce_df['Name'][i] == "Retail Sales ex Autos (MoM)":
        f=ce_df.index[i]
        Autos[f] = ce_df.iloc[i]
        
Autos   = Autos.T
Control = Control.T

df_indicadores = pd.concat([Autos, Control])

In [30]:
Autos   = Autos.reset_index(drop=True)
Control = Control.reset_index(drop=True) 

In [31]:
Autos['Volatility'] = Autos['Volatility'].astype(object).astype(int)
Autos.iloc[:,5:9]   = Autos.iloc[:,5:9].astype(object).astype(float)

Control['Volatility'] = Control['Volatility'].astype(object).astype(int)
Control.iloc[:,5:9]   = Control.iloc[:,5:9].astype(object).astype(float)

In [32]:
def escenario(d):
    d["escenario"]=d["actual"]
    for i in range(len(d)): 
        if d["actual"][i] >= d["consensus"][i] >= d["previous"][i]:
            d["escenario"][i] = "A"
        elif d["actual"][i] >= d["consensus"][i] < d["previous"][i]:
            d["escenario"][i]= "B"
        elif d["actual"][i] < d["consensus"][i] >= d["previous"][i]:
            d["escenario"][i]= "C"
        elif d["actual"][i] < d["consensus"][i] < d["previous"][i]:
            d["escenario"][i]= "D"  
    return d

In [33]:
Autos_esce          = escenario(Autos)
Control_esce        = escenario(Control)
df_indicadores_esce = pd.concat([Autos, Control])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A val

In [34]:
#%% match de nuestros indicadores con el precio
comparacion_autos   = precios_df[precios_df.timestamp.isin(Autos.timestamp)]
comparacion_control = precios_df[precios_df.timestamp.isin(Control.timestamp)]

In [35]:
## comprehension de listas para data frames con 30 mins antes del comunicado y 30 mins despues del comunicado.
autoscom   = [precios_df.iloc[i-6:i+7] for i in comparacion_autos.index]
controlcom = [precios_df.iloc[i-6:i+7] for i in comparacion_autos.index]

In [36]:
Autos_esce['direccion']     = Autos['actual']
Autos_esce['pips alcistas'] = Autos['actual']
Autos_esce['pips bajistas'] = Autos['actual']
Autos_esce['volatilidad']   = Autos['actual']

Control_esce['direccion']     = Control['actual']
Control_esce['pips alcistas'] = Control['actual']
Control_esce['pips bajistas'] = Control['actual']
Control_esce['volatilidad']   = Control['actual']

In [37]:
def df_final(df, periodos):
    for i in range(len(df)):
        df['direccion'][i]     = periodos[i]["close"].iloc[-1] - periodos[i]["open"].iloc[6]
        df['pips alcistas'][i] = periodos[i]["high"].iloc[6:-1].max() - periodos[i]["open"].iloc[0]
        df['pips bajistas'][i] = periodos[i]["open"].iloc[0] - periodos[i]["low"].iloc[6:-1].min()
        df['volatilidad'][i]   = periodos[i]["high"].max()-periodos[i]["low"].min()
    return df

In [45]:
Final_autos   = df_final(Autos_esce, autoscom)
Final_control = df_final(Control_esce, controlcom)
Final_autos

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A valu

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,pips alcistas,pips bajistas,volatilidad
0,2009-01-14 13:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,3,-3.1,-2.5,-1.4,C,-0.00550,0.00174,0.00930,0.01104
1,2009-02-12 13:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,3,1.6,-3.1,-0.3,A,-0.00078,0.00476,-0.00020,0.00580
2,2009-03-12 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,3,1.0,1.6,-0.4,B,-0.00214,0.00357,-0.00008,0.00485
3,2009-04-14 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,3,-1.2,1.0,0.0,D,-0.00147,-0.00181,0.00498,0.00598
4,2009-05-13 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,3,-0.2,-1.2,0.0,C,-0.00058,0.00072,0.00486,0.00660
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2019-05-15 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,2,0.1,1.2,0.7,D,-0.00028,0.00114,0.00019,0.00138
125,2019-06-14 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,2,0.5,0.1,0.3,A,-0.00298,0.00080,0.00279,0.00369
126,2019-07-16 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,2,0.4,0.5,0.1,B,-0.00058,0.00002,0.00124,0.00141
127,2019-08-15 12:30:00+00:00,USD,Retail Sales ex Autos (MoM),United States,2,1.0,0.4,0.4,A,-0.00074,-0.00030,0.00160,0.00206


In [58]:
# contamos ocurrencias por escenario
cont_esc_autos   = Final_autos['escenario'].value_counts()
cont_esc_control = Final_control['escenario'].value_counts()

In [57]:
escenarios_tipo = cont_esc_autos.index
fig = go.Figure([go.Bar(x=escenarios_tipo, y=cont_esc_autos)])
fig.update_layout(title_text='Conteo Escenarios Indicador Retail Sales ex Autos')
fig.show()


In [ ]:
escenarios_tipo = cont_esc_control.index
fig = go.Figure([go.Bar(x=escenarios_tipo, y=cont_esc_control)])
fig.update_layout(title_text='Conteo Escenarios Indicador Retail Sales ex Autos')
fig.show()

### Conclusiones